# Пропущенные значения

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

#### Подготовка данных

In [2]:
data = pd.read_csv("../datasets/melb_data.csv")
data

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.79960,144.99840,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.80790,144.99340,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.80930,144.99440,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.79690,144.99690,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.80720,144.99410,Northern Metropolitan,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,12 Strada Cr,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150.0,...,2.0,2.0,652.0,NaN,1981.0,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392.0
13576,Williamstown,77 Merrett Dr,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016.0,...,2.0,2.0,333.0,133.0,1995.0,NaN,-37.85927,144.87904,Western Metropolitan,6380.0
13577,Williamstown,83 Power St,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016.0,...,2.0,4.0,436.0,NaN,1997.0,NaN,-37.85274,144.88738,Western Metropolitan,6380.0
13578,Williamstown,96 Verdon St,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016.0,...,1.0,5.0,866.0,157.0,1920.0,NaN,-37.85908,144.89299,Western Metropolitan,6380.0


In [3]:
# Целевые значения
y = data.Price

In [4]:
# Для примера оставляем только числовые признаки(features)
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

### Подход 1
#### Удаление столбцов с пропущенными значениями

In [6]:
#  Получаем имена столбцов с пропущенными значениями
col_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
col_with_missing

['Car', 'BuildingArea', 'YearBuilt']

In [7]:
# Удаление столбцов в обучающих и проверочных данных
filtred_X_train = X_train.drop(col_with_missing, axis=1)
filtred_X_valid = X_valid.drop(col_with_missing, axis=1)
print(f"Число столбцов до удаления {len(X_train.columns)}, после {len(filtred_X_train.columns)}")

Число столбцов до удаления 12, после 9


### Подход 2
#### Условное вычисление

In [8]:
# Используем SimpleImputer для замены пропущенных значений средним значением по каждому столбцу
from sklearn.impute import SimpleImputer

In [9]:
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Возвращение удаленных столбцов
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

### Подходу 3 
#### An Extension to Imputation (Расширение условного исчисления)
При таком подходе мы вменяем недостающие значения, как и раньше. И, дополнительно, для каждого столбца с отсутствующими записями в исходном наборе данных мы добавляем новый столбец, который показывает местоположение измененных записей.

В некоторых случаях это значительно улучшит результаты. В других случаях это вообще не помогает.

In [10]:
# Сделайте копию, чтобы избежать изменения исходных данных (при вменении)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

In [11]:
# Для каждого столбца, в котором есть нулевые значения, создается новый столбец
for col in col_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()
X_train_plus["Car_was_missing"]

12796    False
9642     False
3207     False
1698     False
761      False
         ...  
5191     False
13418    False
5390     False
860      False
7270     False
Name: Car_was_missing, Length: 10864, dtype: bool

In [12]:
# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))
imputed_X_train_plus

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,4.0,14.2,3149.0,4.0,2.0,2.0,695.0,160.000000,1970.000000,-37.86127,145.14271,13366.0,0.0,0.0,0.0
1,3.0,14.2,3149.0,3.0,1.0,2.0,810.0,153.999485,1965.104287,-37.86838,145.14664,13366.0,0.0,1.0,1.0
2,2.0,4.6,3122.0,2.0,1.0,1.0,82.0,153.999485,1965.104287,-37.81800,145.02680,11308.0,0.0,1.0,1.0
3,2.0,3.2,3054.0,2.0,1.0,1.0,0.0,76.000000,1975.000000,-37.79020,144.97000,3106.0,0.0,0.0,0.0
4,4.0,13.0,3204.0,4.0,2.0,1.0,292.0,153.999485,1965.104287,-37.91480,145.02430,6795.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,3.0,11.2,3073.0,3.0,1.0,1.0,0.0,153.999485,1930.000000,-37.72170,144.99540,21650.0,0.0,1.0,0.0
10860,2.0,3.6,3068.0,2.0,1.0,1.0,164.0,93.000000,1890.000000,-37.77796,144.98374,6244.0,0.0,0.0,0.0
10861,3.0,2.6,3121.0,3.0,1.0,1.0,383.0,153.999485,1993.000000,-37.82480,144.99760,14949.0,0.0,1.0,0.0
10862,2.0,13.9,3165.0,2.0,2.0,1.0,201.0,79.000000,2005.000000,-37.92930,145.06570,10969.0,0.0,0.0,0.0


In [13]:
# Вменение удалило имена столбцов. Необходимо вернуть их обратно
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns